In [13]:
# coding: utf8

#Import

In [14]:
import os
import sys
import json
import pprint
import urllib3
import pandas as pd
from urllib3 import make_headers
from datetime import datetime, date

#Confinguration

In [15]:
pp = pprint.PrettyPrinter(indent=4)
http = urllib3.PoolManager()
make_headers(accept_encoding=True)

{'accept-encoding': 'gzip,deflate'}

In [16]:
HERE = os.getcwd() + '/'
index = 'lmd'
typeidx = 'demo'
host = os.environ['LMD_ELASTIC_HOST']
PROJECTS_CSV = u'demos.csv' 

#Class Demo

In [17]:
class Demo(object):
    """
    """
    
    def __init__(self, title='title of the demo', desc='...', Type='bonnus', date=date.today(), html=u'index.html'):
        self._id = ''
        
        self.title = title
        self.date = date
        self.source = html
        self.demoType = Type
        self.description = desc
        
        self.tools = []
        self.credits = []
        self.seeAlso = []
        self.score = 1.0
    
    def __addAttributes(self, attr, values):
        for v in values:
            if v not in attr: attr.append(v)
                
    def addTools(self, values):
        self.__addAttributes(self.tools, values)
        
    def addCredits(self, values):
        self.__addAttributes(self.credits, values)
    
    def addSeeAlso(self, values):
        self.__addAttributes(self.seeAlso, values)
                
    @staticmethod
    def linkAlias(field, listsep="|", attrsep=",", na=''):
        tab = field.split(listsep)
        if field == na or len(tab) < 1:
            return list()
        else:
            return map(lambda y: {u'alias': y.split(attrsep)[0],\
                                  u'link': y.split(attrsep)[1]}, tab)
        
        
    @property
    def show(self):
        return {u'title': self.title,
           u'description': self.description,
           u'type': self.demoType,
           u'date': self.date.strftime(u'%Y-%m-%d'),
           u'source': self.source,
           u'tools': self.tools,
           u'credits': self.credits,
           u'seeAlso': self.seeAlso
          }
    
    # TODO: Play with heritage and delete this for project/demo/coworker
    @property
    def showstr(self):
        return json.dumps(self.show)


#Loader

### connection

In [18]:
res = http.request('GET', host)
pp.pprint(json.loads(res.data))

{   u'cluster_name': u'elasticsearch',
    u'name': u'Iron Man',
    u'tagline': u'You Know, for Search',
    u'version': {   u'build_hash': u'72cd1f1a3eee09505e036106146dc1949dc5dc87',
                    u'build_snapshot': False,
                    u'build_timestamp': u'2015-11-18T22:40:03Z',
                    u'lucene_version': u'5.3.1',
                    u'number': u'2.1.0'}}


In [19]:
df = pd.read_csv(HERE + PROJECTS_CSV, sep=';', lineterminator=u'\r')

In [20]:
df = df.fillna(value='')
df.head()

,id,title,description,type,date,source,tools,seeAlso,credits
0,_carcrashone,Paris car accidents 12/13,Which types of vehicule are involved in accide...,graph,20/04/15,cars_one.html,python|spark|D3.js,"map on same subject,/player/_carcrashtwo|#link...","thx to @kentenglish for all the chart tips,htt..."
1,_carcrashtwo,Paris car accidents per rings,Lets have a look the accident distribution in ...,graph,21/04/15,cars_two.html,python|spark|D3.js,"pie on the same subject,/player/_carcrashone|l...",
2,_composite,Design pattern move: Composite,,code,23/06/15,,,,
3,_blank2,a blank demo,X,ml,,,,,
4,_blank,a blank demo2,X,db,,,,,


In [21]:
for idx, p in df.iterrows():
    d = Demo(title=p['title'], Type=p['type'], html=p['source'])
    d.description = p['description']
    d.addTools(p['tools'].split('|'))
    d.addCredits(Demo.linkAlias(p['credits']))
    d.addSeeAlso(Demo.linkAlias(p['seeAlso']))
    d.id = p['id']
    try:
        d.date = datetime.strptime(p[u'date'], u'%d/%m/%y')
    except:
        d.date = datetime.today()
        
    #pp.pprint(d.show)
    http.request('POST','/'.join([host, index, typeidx, p[u'id']]), body=d.showstr)
    

#Update 

### find the document

### Perform update

In [23]:
tools = [u'Spark', u'D3.js']

### Save